In [ ]:

import numpy as np
import pandas as pd 

import os

In [ ]:
import torch
import torchvision
import torchvision.transforms as TT
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split
from PIL import Image

In [ ]:
data_dir='/kaggle/input/sports-classification'

In [ ]:
from torchvision.datasets import ImageFolder
train_ds=ImageFolder(data_dir+'/train',transform=ToTensor())
valid_ds=ImageFolder(data_dir+'/valid',transform=ToTensor())
test_ds=ImageFolder(data_dir+'/test',transform=ToTensor())

In [ ]:
len(train_ds),len(valid_ds),len(test_ds)

In [ ]:
image,label=train_ds[0]
print(image.shape,label)
plt.imshow(image.permute(1,2,0))

In [ ]:
train_dl=DataLoader(train_ds,batch_size=32,shuffle=True,num_workers=3,pin_memory=True)
val_dl=DataLoader(valid_ds,batch_size=32,shuffle=True,num_workers=3,pin_memory=True)

In [ ]:
from torchvision.utils import make_grid
def show_batch(dl):
    for img,label in dl:
        print(img.shape)
        fig,ax=plt.subplots(figsize=(16,16))
        ax.set_xticks([]);ax.set_yticks([])
        ax.imshow(make_grid(img,nrow=8).permute(1,2,0))
        break

show_batch(train_dl)

In [ ]:
def accuracy(output,label):
    _,pred=torch.max(output,dim=1)
    return torch.tensor(torch.sum(pred==label).item()/len(pred))


class ImageClassifierBase(nn.Module):
    def training_step(self,batch):
        image,pred=batch
        out=self(image)
        loss=F.cross_entropy(out,pred)
        return loss

    def validation_step(self,batch):
       image,pred=batch
       out=self(image)
       val_loss=F.cross_entropy(out,pred)
       val_acc=accuracy(out,pred)
       return {'val_loss':val_loss,'val_acc':val_acc}

    def validation_epoch_end(self,output):
       batch_loss=[x['val_loss'] for x in output]
       epoch_loss=torch.stack(batch_loss).mean()
       batch_acc=[x['val_acc'] for x in output]
       epoch_acc=torch.stack(batch_acc).mean()
       return {'val_loss':epoch_loss.item(),'val_acc':epoch_acc.item()}

    def epoch_end(self,epoch,result):
         print("Epoch [{}],{} train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
             epoch, "last_lr: {:.5f},".format(result['lrs'][-1]) if 'lrs' in result else '',
             result['train_loss'], result['val_loss'], result['val_acc']))

    @torch.no_grad()
    def evaluate(self,val_dl):
       self.eval()
       output=[self.validation_step(batch) for batch in val_dl]
       return self.validation_epoch_end(output)

In [ ]:
from torchvision import models

class SportModel(ImageClassifierBase):
    def __init__(self,num_classes,pretrained=True):
        super().__init__()

        self.network=models.resnet34(pretrained=pretrained)
        self.network.fc=nn.Linear(self.network.fc.in_features,num_classes)

    def forward(self,xb):
        return self.network(xb)

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    def __init__(self,dl,device):
        self.dl=dl
        self.device=device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)
           


           
           

In [ ]:
from tqdm.notebook import tqdm

def fit(epochs,lr,model,train_dl,val_dl,opt_func=torch.optim.SGD):

    history=[]
    opt=opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        model.train()
        train_losses=[]

        for batch in tqdm(train_dl):
            loss=model.training_step(batch)
            train_losses.append(lose)
            loss.backward()
            opt.step()
            opt.zero_grad()
        result=model.evaluate(val_loader)
        result['train_losses']=torch.stack(train_losses).mean().item()
        model.epoch_end(epoch,result)
        history.append(result)
    return history


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


In [ ]:
def fit_one_cycle(epochs,max_lr,model,train_dl,val_dl,
                 weight_decay=0,grad_clip=None,
                  opt_func=torch.optim.SGD):

    torch.cuda.empty_cache()
    history=[]

    opt=opt_func(model.parameters(),max_lr,
                 weight_decay=weight_decay)
    sched=torch.optim.lr_scheduler.OneCycleLR(opt,max_lr,
                                             epochs=epochs,
                                             steps_per_epoch=len(train_dl))

    for epoch in range(epochs):
        model.train()
        train_losses=[]
        lrs=[]

        for batch in tqdm(train_dl):
            loss=model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(),
                                         grad_clip)
            opt.step()
            opt.zero_grad()
            lrs.append(get_lr(opt))
            sched.step()
        result=model.evaluate(val_dl)
        result['train_loss']=torch.stack(train_losses).mean().item()
        result['lrs']=lrs
        model.epoch_end(epoch,result)
        history.append(result)
    return history

In [ ]:
device=get_default_device()
device

In [ ]:
train_dl=DeviceDataLoader(train_dl,device)
val_dl=DeviceDataLoader(val_dl,device)

In [ ]:
model=SportModel(len(train_ds.classes))
to_device(model,device);

In [ ]:
history = [model.evaluate(val_dl)]
history

In [ ]:
epochs = 6
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:

history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)

In [ ]:
def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.');
plot_lrs(history)

In [ ]:
def predict(image,model):
    xb=to_device()

In [ ]:
def predict_image(image, model):
    xb = to_device(image.unsqueeze(0), next(model.parameters()).device)  # use model's device
    yb = model(xb)
    _, preds = torch.max(yb, dim=1)
    return train_ds.classes[preds[0].item()]


In [ ]:
image, label = test_ds[0]
plt.imshow(image.permute(1, 2, 0).clamp(0, 1))
print('Label:', test_ds.classes[label], 'Predicted:', predict_image(image, model))


In [ ]:
image, label = test_ds[90]
plt.imshow(image.permute(1, 2, 0).clamp(0, 1))
print('Label:', test_ds.classes[label], 'Predicted:', predict_image(image, model))


In [ ]:
image, label = test_ds[444]
plt.imshow(image.permute(1, 2, 0).clamp(0, 1))
print('Label:', test_ds.classes[label], 'Predicted:', predict_image(image, model))
